## Scraping 


#### Getting a list of subpages for each instrument

Let's get a list of all the available instrument names and webpages for sample banks from the 2012 collection on the University of Iowa Music Department website

In [1]:
import requests 
from bs4 import BeautifulSoup 

base_url = 'http://theremin.music.uiowa.edu/'
req = requests.get(base_url+'MIS.html') 
  
soup = BeautifulSoup(req.content, 'html5lib') 
print(soup.prettify()) 

<html>
 <head>
  <title>
   University of Iowa Electronic Music Studios
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <link href="file:///Sites/Unnamed Site 1/CSS/Level1_Arial.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   <!--
.style1 {
	color: #0000ff;
	font-family: Verdana, Geneva, sans-serif;
}
.style2 {
	font-size: 10.5pt;
	font-family: Verdana, Geneva, sans-serif;
}
.style1 a {
	color: #0000FF;
}
.style1 a {
	font-family: Verdana, Geneva, sans-serif;
	font-size: 10.5pt;
	text-align: center;
}
.ems {
	font-family: Verdana, Geneva, sans-serif;
}
.ems {
	font-family: Verdana, Geneva, sans-serif;
}
.style1 b {
	font-family: Verdana, Geneva, sans-serif;
}
.style3 {
	font-family: Verdana, Geneva, sans-serif;
}
.style1 td .style2 u {
	font-size: 10.5pt;
	text-align: center;
}
.style1 td p .style2 {
	font-size: 10.5px;
}
.hell {
	font-size: 10.5pt;
}
.style4 {
	font-size: 10.5px;
}
.w {
	text-align: left;
}
.bcccc {
	text-align:

In [2]:
instruments = soup.find(id = 'MenuBar1').contents[3].contents[2]
instruments_unfiltered = instruments.contents[3].select('a') #[href="#"]')
instrument_links = list(filter(lambda x: x.attrs['href'][:16] == 'MIS-Pitches-2012', instruments_unfiltered))

In [3]:
instruments = list(map(lambda x: (str(x.contents[0]).strip('()012 '), x.attrs['href']), instrument_links))[:-4]
instruments

[('Flute', 'MIS-Pitches-2012/MISFlute2012.html'),
 ('Alto Flute', 'MIS-Pitches-2012/MISaltoflute2012.html'),
 ('Bass Flute', 'MIS-Pitches-2012/MISBassFlute2012.html'),
 ('Oboe', 'MIS-Pitches-2012/MISOboe2012.html'),
 ('Eb Clarinet', 'MIS-Pitches-2012/MISEbClarinet2012.html'),
 ('Bb Clarinet', 'MIS-Pitches-2012/MISBbClarinet2012.html'),
 ('Bass Clarinet', 'MIS-Pitches-2012/MISBbBassClarinet2012.html'),
 ('Bassoon', 'MIS-Pitches-2012/MISBassoon2012.html'),
 ('Bb Soprano Saxophone', 'MIS-Pitches-2012/MISBbSopranoSaxophone2012.html'),
 ('Eb Alto Saxophone', 'MIS-Pitches-2012/MISEbAltoSaxophone2012.html'),
 ('Horn', 'MIS-Pitches-2012/MISHorn2012.html'),
 ('Bb Trumpet', 'MIS-Pitches-2012/MISBbTrumpet2012.html'),
 ('Tenor Trombone', 'MIS-Pitches-2012/MISTenorTrombone2012.html'),
 ('Bass Trombone', 'MIS-Pitches-2012/MISBassTrombone2012.html'),
 ('Tuba', 'MIS-Pitches-2012/MISTuba2012.html'),
 ('Violin', 'MIS-Pitches-2012/MISViolin2012.html'),
 ('Viola', 'MIS-Pitches-2012/MISViola2012.html'),
 (

We've filterd out unpitched percussion instruments, and cleaned up the instrument names for convenience matching with other sample banks later. There are individual samples for each pitch in the 2012 data set, but it is easier to download the zip files. For many instruments there are multiple zip files for piano, mezzo forte, and forte samples. Some instruments have additional expressions, like tremlolo or pizzacato. We'll have to decide whether to treat these as "the same" instrument, or "different" instruments for our analysis. For now, we will download them all and build in the categorization as an addition feature in our database.

In [4]:
instrument_zips = {}

for instr, rel_url in instruments:
    req = requests.get(base_url + rel_url)
    soup = BeautifulSoup(req.content, 'html5lib') 
    zip_links_unfiltered = soup.find_all('a')
    zip_links = list(filter(lambda x: x.attrs['href'][-4:] == '.zip', zip_links_unfiltered))
    zip_urls = list(map(lambda x: base_url + x.attrs['href'][3:], zip_links))
    instrument_zips[instr] = zip_urls

In [5]:
def download_file(url, target_path):
    print(f'Downloading {url} to {target_path} ...' , end=' ')
    try:
        response = requests.get(url, stream=True)
        handle = open(target_path, "wb")
        for chunk in response.iter_content(chunk_size=512):
            if chunk:
                handle.write(chunk)
        handle.close()
        print('Done.')
    except:
            print("Error downloading:", sys.exc_info()[0])
            raise

In [6]:
from zipfile import ZipFile
from os import remove

def unzip_and_delete_archive(directory, zip_filename, subfolder):
    with ZipFile(directory + zip_filename, 'r') as zipObj:
        print(f'Unzipping {zip_filename} ...', end=' ')
        try:
            zipObj.extractall(directory + subfolder)
            print('Done.')
        except:
            print("Error unzipping:", sys.exc_info()[0])
            raise
    remove(directory + zip_filename)

### Create Database for Samples

In [7]:
import config
import mysql
import mysql.connector
from mysql.connector import errorcode

# DB FUNCTIONS

def connect(db_name=None):
    if db_name:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password,
            database = db_name)
    else:
        cnx = mysql.connector.connect(
            host = config.rds_host,
            user = config.rds_user,
            passwd = config.rds_password)
    cursor = cnx.cursor()
    return cursor, cnx

def create_database_helper(cursor, db_name):
    try:
        cursor.execute(
            f"CREATE DATABASE {db_name} DEFAULT CHARACTER SET 'utf8'"
        )
        print(f"Database {db_name} created successfully.")
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

def create_database(db_name):
    cursor, cnx = connect() 
    try:
        cursor.execute("USE {}".format(db_name))
    except mysql.connector.Error as err:
        print("Database {} does not exist.".format(db_name))
        if err.errno == errorcode.ER_BAD_DB_ERROR:
            create_database_helper(cursor, db_name)
            cnx.database = db_name
        else:
            print(err)
            exit(1)
    cnx.close()

In [9]:
create_database('sound_shift')

Database sound_shift does not exist.
Database sound_shift created successfully.


In [11]:
create_sample_table_query = """CREATE TABLE samples (
    sample_id int NOT NULL AUTO_INCREMENT,
    instrument_name varchar(255) NOT NULL,
    note varchar(255) NOT NULL,
    expression varchar(255),
    source varchar(255),
    file_name varchar(255),
    file_extension varchar(255),
    PRIMARY KEY (sample_id)
    )"""

cur, cnx = connect('sound_shift')
cur.execute(create_sample_table_query)
cnx.close()

### Using AWS S3 for storing audio samples

In [12]:
import logging
import boto3
from botocore.exceptions import ClientError


def upload_file(file_name, bucket='instrument-samples-1337', object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3',
                             aws_access_key_id=config.aws_access_key_id,
                             aws_secret_access_key=config.aws_secret_access_key)
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [16]:
from os.path import basename

def upload_sample(file_name, instrument_name, note, expression, source):
    file_extension = file_name.split('.')[-1] # get file extension
    cur, cnx = connect()
    cur.execute(f"""INSERT INTO sound_shift.samples (instrument_name, note,
                                                     expression, source,
                                                     file_name, file_extension) 
                    VALUES ("{instrument_name}", "{note}",
                            "{expression}", "{source}",
                            "{basename(file_name)}", "{file_extension}");""")
    sample_id = cur.lastrowid
    print(f'Attemping to upload sample {file_name}, id={sample_id} to S3 bucket...', end=' ')
    try:
        upload_file(file_name, object_name=f"{sample_id}.{file_extension}")
        cnx.commit()
        cnx.close()
        print('Success!')
    except:
        print("Upload failed :'(")
        cnx.close()
        raise

    

In [17]:
from os import rmdir, listdir
from shutil import rmtree

def extract_Iowa2012_metadata(file_name):
    left, right = file_name.split('.stereo.')[:2]
    metadata = {'instrument_name' : left.split('.')[0],
                'note' : left.split('.')[-1],
                'expression' : ' '.join(left.split('.')[1:-1]) if len(left.split('.')) > 2 else '',
                'source' : 'Iowa2012',
                'file_extension' : right
                }
    return metadata

def upload_Iowa2012_samples(directory):
    instrument_file_names = list(filter(lambda x: len(x.split('.')) > 4, listdir(directory)))
    metadata = list(map(extract_Iowa2012_metadata, instrument_file_names))
    for i, file_name in enumerate(instrument_file_names):
        upload_sample(directory + file_name,
                      metadata[i]['instrument_name'],
                      metadata[i]['note'],
                      metadata[i]['expression'],
                      metadata[i]['source'])
    rmtree(directory)

Manually create directory ./Iowa2012 before running this next cell.

In [18]:
for inst, zip_urls in instrument_zips.items():
    for url in zip_urls:
        subdirectory_name = inst.replace(' ', '') + '/'
        zip_filename = url.split('/')[-1]
        directory = './Iowa2012/'
        download_file(url, directory + zip_filename)
        unzip_and_delete_archive(directory, zip_filename, subdirectory_name)
        upload_Iowa2012_samples(directory + subdirectory_name)

Unzipping Flute.vib.ff.stereo.zip ... Done.
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.Gb6.stereo.aif, id=1 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.Bb5.stereo.aif, id=2 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.G4.stereo.aif, id=3 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.Eb5.stereo.aif, id=4 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.A6.stereo.aif, id=5 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.F5.stereo.aif, id=6 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.Db5.stereo.aif, id=7 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.C5.stereo.aif, id=8 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Flute/Flute.vib.ff.Ab6.stereo.aif, id=9 to S3 bucket... Success!
Attemping to upload sample ./Iowa

Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.Ab4.stereo.aif, id=145 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.D3.stereo.aif, id=146 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.G4.stereo.aif, id=147 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.A3.stereo.aif, id=148 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.B4.stereo.aif, id=149 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.Gb4.stereo.aif, id=150 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassFlute/BassFlute.ff.C5.stereo.aif, id=151 to S3 bucket... Success!
Unzipping Oboe.ff.stereo.zip ... Done.
Attemping to upload sample ./Iowa2012/Oboe/Oboe.ff.A4.stereo.aif, id=152 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Oboe/Oboe.ff.B3.stereo.aif, id=153 to S3 bucket... Success!
Attempi

Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.Ab5.stereo.aif, id=288 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.E2.stereo.aif, id=289 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.F5.stereo.aif, id=290 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.Bb3.stereo.aif, id=291 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.Eb3.stereo.aif, id=292 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.G4.stereo.aif, id=293 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.D3.stereo.aif, id=294 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.Db5.stereo.aif, id=295 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BassClarinet/BassClarinet.ff.C3.stereo.aif, id

Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.Gb3.stereo.aif, id=423 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.Db5.stereo.aif, id=424 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.E3.stereo.aif, id=425 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.F4.stereo.aif, id=426 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.C4.stereo.aif, id=427 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.Ab3.stereo.aif, id=428 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.Eb5.stereo.aif, id=429 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.ff.Ab4.stereo.aif, id=430 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/EbAltoSaxophone/AltoSax.vib.f

Unzipping Trumpet.novib.ff.stereo.zip ... Done.
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.Ab4.stereo.aif, id=565 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.G5.stereo.aif, id=566 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.F4.stereo.aif, id=567 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.E3.stereo.aif, id=568 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.Gb4.stereo.aif, id=569 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.C4.stereo.aif, id=570 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.B5.stereo.aif, id=571 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpet.novib.ff.A5.stereo.aif, id=572 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/BbTrumpet/Trumpe

...

Unzipping Marimba.roll.ff.stereo.zip ... Done.
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.E6.stereo.aif, id=1786 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.C4.stereo.aif, id=1787 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.Db3.stereo.aif, id=1788 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.Bb6.stereo.aif, id=1789 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.Eb6.stereo.aif, id=1790 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.A2.stereo.aif, id=1791 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.B5.stereo.aif, id=1792 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.Gb5.stereo.aif, id=1793 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Marimba/Marimba.roll.ff.Bb3.ste

Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.B5.stereo.aif, id=1920 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.D7.stereo.aif, id=1921 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.Bb4.stereo.aif, id=1922 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.Gb7.stereo.aif, id=1923 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.A5.stereo.aif, id=1924 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.G7.stereo.aif, id=1925 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.F6.stereo.aif, id=1926 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Xylophone/Xylophone.hardrubber.roll.ff.Db6.stereo.aif, id=1927 to S3 bucket... Succes

Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.Ab4.stereo.aif, id=2048 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.C4.stereo.aif, id=2049 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.E6.stereo.aif, id=2050 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.C3.stereo.aif, id=2051 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.F6.stereo.aif, id=2052 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.Ab3.stereo.aif, id=2053 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.A5.stereo.aif, id=2054 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.sustain.ff.Bb5.stereo.aif, id=2055 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraph

Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.Eb5.stereo.aif, id=2177 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.Bb5.stereo.aif, id=2178 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.D5.stereo.aif, id=2179 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.E4.stereo.aif, id=2180 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.F3.stereo.aif, id=2181 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.Ab3.stereo.aif, id=2182 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.C6.stereo.aif, id=2183 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibraphone.dampen.ff.F6.stereo.aif, id=2184 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Vibraphone/Vibr

Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.G6.stereo.aif, id=2316 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.Eb6.stereo.aif, id=2317 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.Bb6.stereo.aif, id=2318 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.F7.stereo.aif, id=2319 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.E7.stereo.aif, id=2320 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.Ab7.stereo.aif, id=2321 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.C8.stereo.aif, id=2322 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.D6.stereo.aif, id=2323 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crotales/Crotale.ff.Gb7.stereo.aif, id=2324 to S3 bucket... Success!
Attemping to upload sample ./Iowa2012/Crot